# При работе с данными выяснилось, что объектов из классов:
#- товары для детей
#- украшения и аксессуары
#- электроника
#- посуда
#- бытовая техника
#- обувь
#было чрезвычайно мало и ни одна из использованных ранее моделей не могла их найти, поэтому с помощью регулярных выражений были найдены некоторые представители из вышеперечисленных классов и таким образом создался файл valid_126.csv

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import torch
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
train = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/train.csv')
print(train.shape[0])
train.head()

1818


,text
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...


In [36]:
test = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/test.csv')
print(test.shape[0])
test.head()

7276


,text
0,Советую продавца
1,По вашему это платье???? Это узкая кофта !!!! За такие деньги говно!
2,"Жуткая синтетика. Неприятная ткань. Летом не поносить из-за материала. Полно торчащих ниток. Очень неудобная доставка. Лучше бы сделали так, чтобы посылка приходила на почту."
3,"Джемперок так себе на хилую четверку,запах голой синтетики,Цвет грязно синий ,на фото выглядит презентабельнее,скорее всего носить не буду"
4,"обычная х/б рубашка.не плотная,просвечивает нитки не обрезаны,в магазине вряд ли бы такую купила.качество среднее.посылка долго шла по России.пока не стирали."


In [4]:
valid = pd.read_excel('/content/drive/MyDrive/NLP_CASE_FINAL/valid.xlsx')
print(valid.shape)
valid.head()

(100, 2)


,text,labels
0,"Долгая доставка.Плохая обработка швов,везде то...",одежда
1,Продавец мошенник говорил что товар будет отсл...,нет товара
2,Размер не соответствует. Заказали 4XL по факту...,одежда
3,"замечательно, точно как в описании, очень симп...",текстиль
4,описание точное. Качеством довольна: кофточка ...,одежда


In [30]:
from sklearn.model_selection import train_test_split
new_train, valid = train_test_split(
    train,
    random_state=42,
    shuffle = True,
    train_size=1718,
    test_size=100
)

In [31]:
new_train

,text
23,"Цвет черный а не темно синий , обидно. Шапка х..."
906,товар не пришел. затерялся где-то в пути прода...
485,Too long. And a bit too tight in the chest.
1268,Здравствуйте.Шапка не плохая.Тонкая и слабая р...
1629,"Платье так себе. Ткань дешевая, размер XL соот..."
...,...
1130,доставка быстрая продавец молодец.
1294,"Ждала 5 месяцев посылку ,открыла спор, продаве..."
860,"Синтетика, в центре груди проходит шов-не ровн..."
1459,Good goods and fast delivery. Thank you seller.


In [34]:
import re

def find_by_regex(df, regex_pattern, column="text", n=100):
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    mask = df[column].apply(lambda x: bool(pattern.search(str(x))))
    result = df[mask]

    if len(result) > n:
        result = result.sample(n=n, random_state=42)

    return result

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
regex = r"" #сюда вписывал различные регулярные выражения
examples = find_by_regex(new_train, regex, n=100)
examples[["text"]]

,text
291,"Ужасная кофта. Не рекомендую. Маленькая, висит как тряпка."
1195,не пришло деньги не вернули(
1340,"размеры маек очень некорректно отображены в размерной сетке .майки короткие и широкие . подойдут только на низких и полных девушек . материал вискоза , по пошиву претензий нет ."
185,"очень тоненький. цвет не такой как на фото, намного темнее. но за такую цену 565 руб. нормально"
128,Не подошёл размер
1454,Платье соответствует описанию. Доставка быстрая. Пришло с браком - дырочка со стрелкой. Али по спору вернул деньги. С продавцом не общалась.
1012,"Все хорошо, отправили быстро. В подарок носочки ))"
1734,"Комбез как мешок если честно, очень большой+ ещё и ткань тянется"
749,"синтетика сплошная мне не очень понравился. продавец хороший, доставка быстрая"
465,"все как в описании,доставка быстрая, хороший материал."


In [37]:
import re

def find_by_regex(df, regex_pattern, column="text", n=100):
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    mask = df[column].apply(lambda x: bool(pattern.search(str(x))))
    result = df[mask]

    if len(result) > n:
        result = result.sample(n=n, random_state=42)

    return result

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
regex = r"" #сюда вписывал различные регулярные выражения
examples = find_by_regex(test, regex, n=100)
examples[["text"]]

,text
6058,"Товар был заказан 1 марта, через пару недель трекинг показал что товар в Москве.\r\r\nв почтовом отделении не появился, продавец три раза продлял срок защиты покупателя, товар не пришел. по открытому спору вернули средства."
5441,"Очень хорошая блузка. Просто супер. Ткань натуральная. Хороший пошив, красивый цвет и размер подошёл. На ОГ 108 см ОТ 95 см. сидит свободно. С момента заказа до получения - две недели."
4475,"отличная жилетка, но надо брать на размер больше, маломерит"
3689,"Куртка пришла не того размера заказывали S а пришла M, а качество хорошее, доставили быстро."
4515,"Заказ не получили. Продавец общительный, после открытия спора деньги вернул все на следующий день. Продавца рекомендую."
185,"Шла пол месяца. Соответствует описанию, похоже шерсть присутствует потому что колется. Огорчена что с размером не угадала. Ношу хs а заказала м, рукава и длина маловата((((((((((((((((((((((хотя по размерам смотрела"
4894,"ждала пуховик,пришло вот что.\r\nоткрыла спор,жду денег назад."
3258,"Нормально сидит, по размеру идеально.Минус - платье просвечивает.\r\r\nКачество вполне соответствует цене."
4990,очень тонкая ткань. в качестве платья носить нельзя.
6142,на наш 42 xl хорошо.
